In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import datetime as dt

In [4]:
from collections import Counter

In [5]:
import matplotlib.pyplot as plt

In [6]:
from IPython.display import clear_output



### Import Data

In [7]:
XRP_Increase = pd.read_csv('../XRP_5min_Increase_SmallAmount_5min.csv')
XRP_Decrease = pd.read_csv('../XRP_5min_Decrease_SmallAmount_5min.csv')

In [8]:
original_columns = ['datetime_real','date','close','high','low','open','quoteVolume','volume','weightedAverage']

In [9]:
XRP_Increase.head()

,Unnamed: 0,Unnamed: 0.1,close,date,high,low,open,quoteVolume,volume,weightedAverage,...,Increase_0.1_percent,Increase_0.2_percent,Increase_0.3_percent,Increase_0.4_percent,Increase_0.5_percent,Increase_0.6_percent,Increase_0.8_percent,Increase_1.0_percent,Increase_1.2_percent,Increase_1.5_percent
0,12736,12736,0.069288,1493875200,0.069960,0.069288,0.069960,163.538132,11.431388,0.069900,...,1493875500,1493875500,1493875500,1493875500,1493875500,1493875500,1493875500,1493881200,1493881500,1493881500
1,12737,12737,0.069100,1493875500,0.069949,0.069000,0.069949,113714.544855,7909.709770,0.069558,...,1493875800,1493876700,1493876700,1493876700,1493876700,1493877000,1493881200,1493881200,1493881200,1493881500
2,12738,12738,0.068000,1493875800,0.069100,0.067500,0.069000,220670.240024,15126.256548,0.068547,...,1493876100,1493876100,1493876100,1493876100,1493876100,1493876100,1493876100,1493876100,1493876100,1493876400
3,12739,12739,0.068465,1493876100,0.068465,0.066432,0.067800,271144.858354,18298.825581,0.067487,...,1493876400,1493876400,1493876400,1493876400,1493876400,1493876400,1493876400,1493876400,1493876400,1493876400
4,12740,12740,0.068646,1493876400,0.068646,0.067517,0.067802,595249.214904,40678.358104,0.068338,...,1493876700,1493876700,1493876700,1493876700,1493876700,1493876700,1493876700,1493876700,1493876700,1493876700


In [10]:
np.array( [1,2,3,4,5,6])*0.1

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])

In [11]:
# for n in np.array( [1,2,3,4,5,6,8,10])*0.1:
#     XRP_Increase['shift_open_{}'.format(n)] = XRP_Increase['Increase_{}_percent'.format(n)].shift(1)

for 0.6% incraese:

how many continous increase are there?

In [12]:
# XRP_Increase['diff_0.6'] = XRP_Increase['shift_open_0.6000000000000001'] - XRP_Increase['Increase_0.6000000000000001_percent']

In [13]:
#(XRP_Increase[XRP_Increase['diff_0.6'] != 0]['diff_0.6'].value_counts())/(len(XRP_Increase))

In [14]:
n = 6*0.1

## Adjust for waiting

In [15]:
for col in ['Unnamed: 0','Unnamed: 0.1']:
    try:
        del XRP_Increase[col]
    except:
        None

In [16]:
XRP_Analysis_table = XRP_Increase

In [17]:
def brute_force_finding(drop_limit, stop_loss_limit, sell_limit, RH_time, data_to_use, timing_stop = 31536000):
    """     
    inputs:
            drop_limit
            stop_loss_limit
            sell_limit
            RH_time
            XRP_Analysis_table
            
            timing_stop (options), defult = 31536000 ( 1 year time )
            
    """
    
    time_event = {}
    starting_asset = 50
    bought_crypto = False
    gold_available = starting_asset 
    XRP_amount = 0


#     drop_limit = 0.005 # used mean on this side, actual 0.004686, but decreased abit for optimistic view 
#     stop_loss_limit = 0.01
#     sell_limit = 0.006
#     RH_time = '2H_RA_on_Close'


    "inputs"
    drop_limit         = drop_limit
    stop_loss_limit    = stop_loss_limit
    sell_limit         = sell_limit
    RH_time            = RH_time
    XRP_Analysis_table = data_to_use
    timing_stop        = timing_stop
    
    
    if RH_time not in data_to_use.columns:
        print("check columns of data imported, column of RH attempted : {}".format(RH_time))
        return "Wrong data input"
    

    while gold_available >= 0:

        #for ind, df in XRP_Analysis_table[XRP_Analysis_table['datetime_real'] > '2017-07-07'].iterrows():
        for ind, df in XRP_Analysis_table.iterrows():
    #         print(df['date'])

            if bought_crypto != True: # then looking to buy at drop limit or lower
                if ( df[RH_time] - df['low'] ) / df[RH_time] >= drop_limit:
                    "then buy crypto"

                    ## might add here a waiting time!!! ###

                    XRP_amount += (gold_available/df['low']) * 0.998 # due to 0.2% fee

                    gold_available -= gold_available

                    time_event.update({df['date']  : ( gold_available, XRP_amount , 'brought' , df['low'])})

                    last_xrp_traction_price =  df['low']

                    bought_crypto = True
                    
                    time_brought = df['date'] 
    #                 print('last_xrp_traction_price ', last_xrp_traction_price)

            else:
                if ( df[RH_time] - df['low'] ) / df[RH_time] >= stop_loss_limit: # then hit stop loss, force sell
#                     print('stoppppppppppppppppppppppppp')
                    gold_available += XRP_amount * df['low'] * 0.998  # sell at low price

                    XRP_amount -= XRP_amount

                    time_event.update({df['date']  : ( gold_available, XRP_amount , 'stop_loss_hit' ,  df['low'])})

                    bought_crypto = False

                # Time limit, might not need
#                 " other limits needed - such as RA "
                elif df['date'] - time_brought >= timing_stop: # then time limit reached, sell

                    #print(last_xrp_traction_price)

                    gold_available += XRP_amount * df['high'] * 0.998  # sell at high price

                    XRP_amount -= XRP_amount

                    time_event.update({df['date']  : ( gold_available, XRP_amount , 'time_stop' , df['close'])})

                    bought_crypto = False

                elif (df['high'] - last_xrp_traction_price)/last_xrp_traction_price >= sell_limit: # then hit target price, sell for profit

#                     print(last_xrp_traction_price)

                    gold_available += XRP_amount * df['high'] * 0.998  # sell at high price

                    XRP_amount -= XRP_amount

                    time_event.update({df['date']  : ( gold_available, XRP_amount , 'sell_profit' , df['high'])})

                    bought_crypto = False                    
                    
                    

        break

    else:
        print('Negative asset: bankrupt  ->>>   ',  gold_available)
        
    transation_log = pd.DataFrame.from_dict(time_event, orient='index', columns= ['Gold', 'XRP_Amount', 'action', 'price_action_happened'])
    transation_log['Gold'] = round(transation_log['Gold'], 2)
    transation_log['XRP_Amount'] = round(transation_log['XRP_Amount'], 4)
    
#     print('end Gold amount ' , transation_log.iloc[-1]['Gold'])
    return (transation_log.iloc[-1]['Gold'] , transation_log)

In [18]:
XRP_Analysis_table['20min_RA_on_Close'] = XRP_Analysis_table['close'].rolling(4).mean().shift(1)
XRP_Analysis_table['1H_RA_on_Close'] = XRP_Analysis_table['close'].rolling(12).mean().shift(1)
XRP_Analysis_table['2H_RA_on_Close'] = XRP_Analysis_table['close'].rolling(24).mean().shift(1)
XRP_Analysis_table['4H_RA_on_Close'] = XRP_Analysis_table['close'].rolling(48).mean().shift(1)
XRP_Analysis_table['12H_RA_on_Close'] = XRP_Analysis_table['close'].rolling(144).mean().shift(1)

In [42]:
ra_list            =    ['20min_RA_on_Close', '1H_RA_on_Close', '2H_RA_on_Close', '4H_RA_on_Close', '12H_RA_on_Close']
buy_in_list        =    np.arange(0.002,0.022, 0.002)
stop_loss_list     =    np.arange(0.01,0.03, 0.002)
time_waiting_list  =    [2, 5, 8, 12, 16, 24] # in hours
sell_limit_list    =    np.arange(0.002,0.032, 0.002)


## reversing

In [43]:
ra_list.reverse()
time_waiting_list.reverse()

buy_in_list = buy_in_list[::-1]
stop_loss_list = stop_loss_list[::-1]
sell_limit_list = sell_limit_list[::-1]

In [44]:
dates_cors = {  1507374600: 1511274300,
                1512774600: 1515174000,
                1515174300: 1518174000,
                1547873100: 1550872800}

In [45]:
for time in dates_cors.items():
    print(time[0], time[1])

1507374600 1511274300
1512774600 1515174000
1515174300 1518174000
1547873100 1550872800


In [46]:
from IPython.display import clear_output

In [47]:
event_dictionary = [(   'buy_in',
                        'stop_loss',
                        'sell_limit',
                        'time_waiting',
                        'ra_column',
                        'time',
                        'buyin',
                        'sell_profit',
                        'stop_loss_hit',
                        'time_stop',
                        'final amount')]

n = 0

for buy_in_input in buy_in_list:
    
    for stop_loss_input in stop_loss_list:
        
        for sell_limit_input in sell_limit_list:
            
            for time_waiting_input in time_waiting_list:
                
                for ra in ra_list:
                    
                    for time in dates_cors.items():
                        n += 1
                        
                        if n%5 == 0:
                            clear_output()
                        
                        print('------------------------------------------------------------------------------------------')
                        print( str(round((n/180000) * 100,5)) + '% progress, for 180,000 iterations')
                        

                        
                        temp = brute_force_finding(drop_limit      =  buy_in_input,
                                            stop_loss_limit =  stop_loss_input,
                                            sell_limit      =  sell_limit_input,
                                            RH_time         =  ra,
                                            data_to_use     =  XRP_Analysis_table[(XRP_Analysis_table['date'] >= time[0] ) &
                                                                                  (XRP_Analysis_table['date'] <= time[1] )],
                                            
                                            timing_stop     =  time_waiting_input*60*60
                                           )
                        
                        event_dictionary.append((buy_in_input,
                                                 stop_loss_input,
                                                 sell_limit_input,
                                                 ra,
                                                 time[0],
                                                 time[1],
                                                 len(temp[1][temp[1]['action'] == 'brought']),
                                                 len(temp[1][temp[1]['action'] == 'sell_profit']),
                                                 len(temp[1][temp[1]['action'] == 'stop_loss_hit']),
                                                 len(temp[1][temp[1]['action'] == 'time_stop']),
                                                 sum(temp[1][-2:]['Gold']) # append last gold value (append df take too much data)
                                                ))
                        
                        
                        print('buy_in at:', buy_in_input , '; ',
                              'stop_loss at:', stop_loss_input , '; ',
                              'sell_limit at:', sell_limit_input , '; ',
                              'time_waiting at:', time_waiting_input , ' Hours; ',
                              'ra at:', ra , '; ',
                              'time at:', time[0], '-' , time[1] , '\n',
                              'buyin:         ' , len(temp[1][temp[1]['action'] == 'brought']) , '\n',
                              'sell_profit:   ', len(temp[1][temp[1]['action'] == 'sell_profit']) , '\n',
                              'stop_loss_hit: ' , len(temp[1][temp[1]['action'] == 'stop_loss_hit']) , '\n',
                              'time_stop:     ' , len(temp[1][temp[1]['action'] == 'time_stop']) , '\n',
                              'final amount:  ' ,  sum(temp[1][-2:]['Gold']) )



------------------------------------------------------------------------------------------
51.94444% progress, for 180,000 iterations
buy_in at: 0.01 ;  stop_loss at: 0.026000000000000002 ;  sell_limit at: 0.002 ;  time_waiting at: 24  Hours;  ra at: 20min_RA_on_Close ;  time at: 1547873100 - 1550872800 
 buyin:          56 
 sell_profit:    54 
 stop_loss_hit:  1 
 time_stop:      1 
 final amount:   56.73
------------------------------------------------------------------------------------------
51.945% progress, for 180,000 iterations
buy_in at: 0.01 ;  stop_loss at: 0.026000000000000002 ;  sell_limit at: 0.002 ;  time_waiting at: 16  Hours;  ra at: 12H_RA_on_Close ;  time at: 1507374600 - 1511274300 
 buyin:          1588 
 sell_profit:    936 
 stop_loss_hit:  652 
 time_stop:      0 
 final amount:   22.46
------------------------------------------------------------------------------------------
51.94556% progress, for 180,000 iterations
buy_in at: 0.01 ;  stop_loss at: 0.02600000

KeyboardInterrupt: 

In [54]:
reverse_table_save = pd.DataFrame(event_dictionary)

In [55]:
# reverse_table_save_1 = reverse_table_save.copy()

In [73]:
reverse_table_save.columns = reverse_table_save.iloc[0].values

In [76]:
reverse_table_save[1:].to_pickle('reverse_save_xrp_brute.pkl')

In [78]:
reverse_table_save = reverse_table_save[1:]

In [80]:
reverse_table_save['buy_in'].unique()

array([0.020000000000000004, 0.018000000000000002, 0.016, 0.014, 0.012,
       0.01], dtype=object)

In [81]:
buy_in_list

array([0.02 , 0.018, 0.016, 0.014, 0.012, 0.01 , 0.008, 0.006, 0.004,
       0.002])

In [53]:
def plotting_view(xrp_data_input , trading_log ):

#     ploting_breakdown = XRP_Analysis_table[(XRP_Analysis_table['date'] >= 1507374600 ) &
#                                            (XRP_Analysis_table['date'] <= 1511274300 )]


    transaction_display = trading_log
    transaction_display['time_start'] = transaction_display.index
    transaction_display['time_end'] = round(transaction_display['time_start'].shift(-1), 0)

    transaction_display['action_to_color'] = transaction_display['action'].shift(-1)

    transaction_display['colour_section'] = transaction_display['action_to_color'].apply(lambda x: 'r' if x == 'stop_loss_hit' else 'g')
    
    ploting_breakdown = xrp_data_input
    plt.figure(figsize= (20,10))


    tran_y_low = ploting_breakdown['low']
    tran_y_high = ploting_breakdown['high']

    tran_y_ra = ploting_breakdown['2H_RA_on_Close']

    tran_x = ploting_breakdown['date']


    plt.plot(tran_x, tran_y_low)
    plt.plot(tran_x, tran_y_high)

    plt.plot(tran_x, tran_y_ra)

    graph_min = ploting_breakdown['low'].min() * 0.9
    graph_max = ploting_breakdown['high'].max() * 1.1

    # plotting transaction
    for ind, value in transaction_display[ (transaction_display['time_start'] <= ploting_breakdown['date'].max()) &
                                           (transaction_display['time_start'] >= ploting_breakdown['date'].min()) &                      
                                           (transaction_display['action'] == 'brought')].iterrows():

        plt.axvspan(value['time_start'], value['time_end'], alpha = 0.1 , facecolor = value['colour_section'])

    #     plt.vlines(value['time_start'], graph_min, graph_max, alpha = 0.4, color = value['colour_section'])
    #     plt.vlines(value['time_end'], graph_min, graph_max, alpha = 0.4, color = value['colour_section'], linestyle = 'dashed')


    # for ind, value in ploting_breakdown[ploting_breakdown['Increase_0.5_percent'] <= ploting_breakdown['date'].max()].iterrows():
    # #     print(value['Increase_0.5_percent'])
    # #     plt.vlines(value['date'], 0, 1, alpha = 0.2, color = 'b',linestyle = 'dashed')
    #     plt.vlines(value['Increase_0.5_percent'], 0, 1, alpha = 0.2, color = 'b',linestyle = 'dashed')


    plt.ylim(graph_min, graph_max )

    plt.xlim(ploting_breakdown['date'].min(), ploting_breakdown['date'].max())
    plt.legend()